# Plot XRD maps from flyscans

This notebook will plot an XRD map from a flyscan. It requires data from the `.h5` masterfile and the `_pilatus_integrated.h5` file.

A ROI for the XRD map (in radial bins) should be defined in one of the cell below.

### Load data and plot average XRD for the map

In [ ]:
%matplotlib widget
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM

In [ ]:
# Define scan location:
#Note that scans must be a list!
scans = [XXX]
proposal,visit=DM.getCurrentProposal()

#Load data
xx, yy, xrd_map, x_xrd, Q = DM.stitchScans(scans,XRF=False,proposal=proposal,visit=visit)
#Apply I0 correction
xrd_map = (xrd_map.transpose(2,0,1)/ I0_map).transpose(1,2,0)
xrd_avg = np.mean(xrd_map,axis=(0,1))


### Plot avarage XRD map

In [ ]:
#Set ROIs:

# define ROI dictionary with a label str and a tuple of lower and upper limits, given in the scattering units (2theta or Q)
#     'label': (lower,upper)
rois = {'peak_1' : (  4, 4.1),
        'peak_2' : (14.24, 16.36)}

# plot average XRD pattern
plt.figure()
plt.title('Average XRD pattern')
plt.plot(x_xrd,xrd_avg)
print(xrd_map.dtype)
if Q:
    plt.xlabel('Q (A-1)')
else:
    plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.yscale('log')
for key in rois:
    plt.axvline(rois[key][0],c='k',ls='--',lw=1)
    plt.axvline(rois[key][1],c='k',ls='--',lw=1)

### Plot XRD map
It is probably nescessesary to change the vmax value

In [ ]:
# Set the number of columns for the figure
cols = 4 

rows = int(len(rois)/cols) + (len(rois)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

I_xrd = []
for i,key in enumerate(rois):
    lower, upper = rois[key]
    lower, upper = np.digitize(lower, x_xrd), np.digitize(upper, x_xrd)
    
    #calculate background
    bkg = np.mean([np.mean(xrd_map[:,:,lower-5:lower],axis=2), np.mean(xrd_map[:,:,upper+1:upper+6],axis=2)])
    # calculate the average diffraction peak signal from the integrated XRD data
    I_xrd.append(np.trapz(xrd_map[:,:,lower:upper] - bkg,x_xrd[lower:upper],axis=2))
    

    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(xx,
                        yy,
                        I_xrd[i],
                        norm='log',
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel('x mm')
    ax.set_ylabel('y mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(rois)+1):
    fig.delaxes(axs[-i])